In [1]:
import numpy as np
import os
import torch
from torchvision import transforms
from tqdm import tqdm

import pandas as pd

import PIL

import matplotlib.pyplot as plt


In [12]:
import open_clip
from open_clip import tokenizer
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')

In [13]:
path = "/cluster/scratch/nkoisheke/artstation_art_data_merged.csv"
text = []
i = 0
artist_names = []
with open(path, 'r') as f:
    for r in f:
        if i > 0:
            artist_names.append(r.split("\t")[0])
        i += 1

In [14]:
print(len(artist_names))

7618


In [23]:
data = []
new_artist_names = []
# change the image folders here
path = "/cluster/scratch/nkoisheke/artworks"
number_of_images = len(os.listdir(path))
for i in tqdm(range(len(artist_names))):
    temp_path = os.path.join(path, f"index_{i}.png") 
    if os.path.exists(temp_path):
        name = artist_names[i]
        if name.isascii():
            img = PIL.Image.open(temp_path)
            temp_img = img.copy()
            img.close()
            temp_img = preprocess(temp_img)
            data.append(temp_img)
            # new_artist_names.append(name)
            # data.append(temp_path)

100%|██████████| 7618/7618 [00:03<00:00, 2212.76it/s]


In [32]:
# img_paths = [d.split("/")[-1] for d in data]
# import pandas as pd

# new_df = pd.DataFrame(
#     {
#         "uid": [i for i in range(len(new_artist_names))],
#         "artist": new_artist_names,
#         "image_name": img_paths,
#     }
# )

# new_df.to_csv("/cluster/scratch/nkoisheke/real_images_artstation_filtered.csv", index=False)

In [17]:
device = torch.device('cuda')
model.to(device);

In [18]:
image_features = []
batch_size = 512

with torch.no_grad():
    for i in tqdm(range(0, len(data), batch_size)):
        batch = []
        for j in range(i, min(i+batch_size, len(data))):
            batch.append(data[j])
        batch = torch.tensor(np.stack(batch))
        batch = batch.to(device)
        img_feats = model.encode_image(batch).float()
        image_features.append(img_feats)
image_features_torch = torch.concatenate(image_features).cpu()

100%|██████████| 14/14 [00:06<00:00,  2.32it/s]


In [19]:
print(image_features_torch.shape)

torch.Size([7031, 512])


In [20]:
# del data

In [21]:
# torch.save(
#     image_features_torch, 
#     '/cluster/scratch/nkoisheke/real_images_artstation_filtered_' + 'laion2b_s34b_b79k' + '_ViT-B-32' + '.pt')

In [51]:
artists = list(set(new_artist_names))
text_features = []

with torch.no_grad():
    for artist in artists:
        prompt = f"Done in the style of {artist} on ArtStation."
        text_tokens = tokenizer.tokenize(prompt)
        text_tokens = text_tokens.to(device)
        txt_feat = model.encode_text(text_tokens).float()
        text_features.append(txt_feat)
text_features_torch = torch.concatenate(text_features).cpu()

In [52]:
idx2artist = {}
artist2idx = {}
for i, artist in enumerate(artists):
    idx2artist[i] = artist
    artist2idx[artist] = i

In [39]:
image_features_torch /= image_features_torch.norm(dim=-1, keepdim=True)
text_features_torch /= text_features_torch.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features_torch @ text_features_torch.T).softmax(dim=-1)
top_probs, top_k_labels = text_probs.cpu().topk(5, dim=-1)

In [40]:
gt_labels = torch.tensor([artist2idx[x] for x in new_artist_names])

top_one_labels = top_k_labels[:, 0]

print(gt_labels.shape, top_one_labels.shape)

correct = (gt_labels == top_one_labels).sum()
print(f"Top 1 score is {round((correct / gt_labels.shape[0]).item() * 100, 2)}")


topk_correct = 0
for i in range(5):
    top_one_labels = top_k_labels[:, i]
    correct = (gt_labels == top_one_labels).sum()
    topk_correct += correct
print(f"Top 5 score is {round((topk_correct / gt_labels.shape[0]).item() * 100, 2)}")
# correct = (gt_labels.repeat() == top_k_labels).sum()
# print(correct)

torch.Size([7031]) torch.Size([7031])
Top 1 score is 2.03
Top 5 score is 6.0
